In [163]:
import numpy as np 
import pandas as pd
import tensorflow as tf

In [164]:
try:
    path = get_file('kddcup.data_10_percent.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz')
except:
    print('Error downloading')
    raise
    
print(path) 

C:\Users\ngohuy143\.keras\datasets\kddcup.data_10_percent.gz


In [165]:
df = pd.read_csv(path, header=None)

In [166]:
print("Read {} rows.". format(len(df)))
# df = df.sample(frac=0.1, replace=False) # sample only 10% of the dataset

df.dropna(inplace=True, axis=1) # for now, just drop NA's (rows with missing values)

# the csv file has no column heads, just add them 

df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

# display 5 rows
df[0:5]

Read 494021 rows.


duration protocol_type service flag  src_bytes  dst_bytes  land  \
0         0           tcp    http   SF        181       5450     0   
1         0           tcp    http   SF        239        486     0   
2         0           tcp    http   SF        235       1337     0   
3         0           tcp    http   SF        219       1337     0   
4         0           tcp    http   SF        217       2032     0   

   wrong_fragment  urgent  hot   ...     dst_host_srv_count  \
0               0       0    0   ...                      9   
1               0       0    0   ...                     19   
2               0       0    0   ...                     29   
3               0       0    0   ...                     39   
4               0       0    0   ...                     49   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                     1.0                     0.0   
1                     1.0                     0.0   
2                     1.0                     0.0   
3                     1.0                     0.0   
4                     1.0                     0.0   

   dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
0                         0.11                          0.0   
1                         0.05                          0.0   
2                         0.03                          0.0   
3                         0.03                          0.0   
4                         0.02                          0.0   

   dst_host_serror_rate  dst_host_srv_serror_rate  dst_host_rerror_rate  \
0                   0.0                       0.0                   0.0   
1                   0.0                       0.0                   0.0   
2                   0.0                       0.0                   0.0   
3                   0.0                       0.0                   0.0   
4                   0.0                       0.0                   0.0   

   dst_host_srv_rerror_rate  outcome  
0                       0.0  normal.  
1                       0.0  normal.  
2                       0.0  normal.  
3                       0.0  normal.  
4                       0.0  normal.  

[5 rows x 42 columns]

## Analyzing the dataset

In [167]:
ENCODING = 'utf-8'
def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [168]:
# analyze our dataset
import os
from sklearn import metrics
from scipy.stats import zscore

analyze(df)


494021 rows
** duration:2495 (0%)
** protocol_type:[icmp:57.41%,tcp:38.47%,udp:4.12%]
** service:[ecr_i:56.96%,private:22.45%,http:13.01%,smtp:1.97%,other:1.46%,domain_u:1.19%,ftp_data:0.96%,eco_i:0.33%,ftp:0.16%,finger:0.14%,urp_i:0.11%,telnet:0.1%,ntp_u:0.08%,auth:0.07%,pop_3:0.04%,time:0.03%,csnet_ns:0.03%,remote_job:0.02%,imap4:0.02%,gopher:0.02%,discard:0.02%,domain:0.02%,iso_tsap:0.02%,systat:0.02%,shell:0.02%,echo:0.02%,rje:0.02%,whois:0.02%,sql_net:0.02%,printer:0.02%,nntp:0.02%,courier:0.02%,mtp:0.02%,netbios_ssn:0.02%,sunrpc:0.02%,vmnet:0.02%,klogin:0.02%,uucp:0.02%,bgp:0.02%,uucp_path:0.02%,nnsp:0.02%,supdup:0.02%,ssh:0.02%,hostnames:0.02%,login:0.02%,efs:0.02%,daytime:0.02%,netbios_ns:0.02%,link:0.02%,pop_2:0.02%,ldap:0.02%,http_443:0.02%,exec:0.02%,netbios_dgm:0.02%,kshell:0.02%,name:0.02%,ctf:0.02%,netstat:0.02%,Z39_50:0.02%,IRC:0.01%,urh_i:0.0%,X11:0.0%,tim_i:0.0%,tftp_u:0.0%,pm_dump:0.0%,red_i:0.0%]
** flag:[SF:76.6%,S0:17.61%,REJ:5.44%,RSTR:0.18%,RSTO:0.12%,SH:0.02%,S

** dst_host_serror_rate:[0.0:80.93%,1.0:17.56%,0.01:0.74%,0.02:0.2%,0.03:0.09%,0.09:0.05%,0.04:0.04%,0.05:0.04%,0.07:0.03%,0.08:0.03%,0.06:0.02%,0.14:0.02%,0.15:0.02%,0.11:0.02%,0.13:0.02%,0.16:0.02%,0.1:0.02%,0.12:0.01%,0.18:0.01%,0.25:0.01%,0.2:0.01%,0.17:0.01%,0.33:0.01%,0.99:0.01%,0.19:0.01%,0.31:0.01%,0.27:0.01%,0.5:0.0%,0.22:0.0%,0.98:0.0%,0.35:0.0%,0.28:0.0%,0.53:0.0%,0.24:0.0%,0.96:0.0%,0.3:0.0%,0.26:0.0%,0.97:0.0%,0.29:0.0%,0.94:0.0%,0.42:0.0%,0.32:0.0%,0.56:0.0%,0.55:0.0%,0.95:0.0%,0.6:0.0%,0.23:0.0%,0.93:0.0%,0.34:0.0%,0.85:0.0%,0.89:0.0%,0.21:0.0%,0.92:0.0%,0.58:0.0%,0.43:0.0%,0.9:0.0%,0.57:0.0%,0.91:0.0%,0.49:0.0%,0.82:0.0%,0.36:0.0%,0.87:0.0%,0.45:0.0%,0.62:0.0%,0.65:0.0%,0.46:0.0%,0.38:0.0%,0.61:0.0%,0.47:0.0%,0.76:0.0%,0.81:0.0%,0.54:0.0%,0.64:0.0%,0.44:0.0%,0.48:0.0%,0.72:0.0%,0.39:0.0%,0.52:0.0%,0.51:0.0%,0.67:0.0%,0.84:0.0%,0.73:0.0%,0.4:0.0%,0.69:0.0%,0.79:0.0%,0.41:0.0%,0.68:0.0%,0.88:0.0%,0.77:0.0%,0.75:0.0%,0.7:0.0%,0.8:0.0%,0.59:0.0%,0.71:0.0%,0.37:0.0%,0.86:0.0

## Encoding the feature vector


In [169]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [170]:
encode_numeric_zscore(df, 'duration')
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'src_bytes')
encode_numeric_zscore(df, 'dst_bytes')
encode_text_dummy(df, 'land')
encode_numeric_zscore(df, 'wrong_fragment')
encode_numeric_zscore(df, 'urgent')
encode_numeric_zscore(df, 'hot')
encode_numeric_zscore(df, 'num_failed_logins')
encode_text_dummy(df, 'logged_in')
encode_numeric_zscore(df, 'num_compromised')
encode_numeric_zscore(df, 'root_shell')
encode_numeric_zscore(df, 'su_attempted')
encode_numeric_zscore(df, 'num_root')
encode_numeric_zscore(df, 'num_file_creations')
encode_numeric_zscore(df, 'num_shells')
encode_numeric_zscore(df, 'num_access_files')
encode_numeric_zscore(df, 'num_outbound_cmds')
encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')
encode_numeric_zscore(df, 'count')
encode_numeric_zscore(df, 'srv_count')
encode_numeric_zscore(df, 'serror_rate')
encode_numeric_zscore(df, 'srv_serror_rate')
encode_numeric_zscore(df, 'rerror_rate')
encode_numeric_zscore(df, 'srv_rerror_rate')
encode_numeric_zscore(df, 'same_srv_rate')
encode_numeric_zscore(df, 'diff_srv_rate')
encode_numeric_zscore(df, 'srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_count')
encode_numeric_zscore(df, 'dst_host_srv_count')
encode_numeric_zscore(df, 'dst_host_same_srv_rate')
encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
encode_numeric_zscore(df, 'dst_host_serror_rate')
encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
encode_numeric_zscore(df, 'dst_host_rerror_rate')
encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]
# This is the numeric feature vector, as it goes to the neural net

duration  src_bytes  dst_bytes  wrong_fragment    urgent       hot  \
0 -0.067792  -0.002879   0.138664        -0.04772 -0.002571 -0.044136   
1 -0.067792  -0.002820  -0.011578        -0.04772 -0.002571 -0.044136   
2 -0.067792  -0.002824   0.014179        -0.04772 -0.002571 -0.044136   
3 -0.067792  -0.002840   0.014179        -0.04772 -0.002571 -0.044136   
4 -0.067792  -0.002842   0.035214        -0.04772 -0.002571 -0.044136   

   num_failed_logins  num_compromised  root_shell  su_attempted  \
0          -0.009782        -0.005679   -0.010552     -0.004676   
1          -0.009782        -0.005679   -0.010552     -0.004676   
2          -0.009782        -0.005679   -0.010552     -0.004676   
3          -0.009782        -0.005679   -0.010552     -0.004676   
4          -0.009782        -0.005679   -0.010552     -0.004676   

         ...         flag-S3  flag-SF  flag-SH  land-0  land-1  logged_in-0  \
0        ...               0        1        0       1       0            0   
1        ...               0        1        0       1       0            0   
2        ...               0        1        0       1       0            0   
3        ...               0        1        0       1       0            0   
4        ...               0        1        0       1       0            0   

   logged_in-1  is_host_login-0  is_guest_login-0  is_guest_login-1  
0            1                1                 1                 0  
1            1                1                 1                 0  
2            1                1                 1                 0  
3            1                1                 1                 0  
4            1                1                 1                 0  

[5 rows x 121 columns]

In [171]:
df[0:5]

duration  src_bytes  dst_bytes  wrong_fragment    urgent       hot  \
0 -0.067792  -0.002879   0.138664        -0.04772 -0.002571 -0.044136   
1 -0.067792  -0.002820  -0.011578        -0.04772 -0.002571 -0.044136   
2 -0.067792  -0.002824   0.014179        -0.04772 -0.002571 -0.044136   
3 -0.067792  -0.002840   0.014179        -0.04772 -0.002571 -0.044136   
4 -0.067792  -0.002842   0.035214        -0.04772 -0.002571 -0.044136   

   num_failed_logins  num_compromised  root_shell  su_attempted  \
0          -0.009782        -0.005679   -0.010552     -0.004676   
1          -0.009782        -0.005679   -0.010552     -0.004676   
2          -0.009782        -0.005679   -0.010552     -0.004676   
3          -0.009782        -0.005679   -0.010552     -0.004676   
4          -0.009782        -0.005679   -0.010552     -0.004676   

         ...         flag-S3  flag-SF  flag-SH  land-0  land-1  logged_in-0  \
0        ...               0        1        0       1       0            0   
1        ...               0        1        0       1       0            0   
2        ...               0        1        0       1       0            0   
3        ...               0        1        0       1       0            0   
4        ...               0        1        0       1       0            0   

   logged_in-1  is_host_login-0  is_guest_login-0  is_guest_login-1  
0            1                1                 1                 0  
1            1                1                 1                 0  
2            1                1                 1                 0  
3            1                1                 1                 0  
4            1                1                 1                 0  

[5 rows x 121 columns]

In [172]:
# convert to numpy - classification
x_columns = df.columns.drop('outcome')
x = df[x_columns].values
dummies = pd.get_dummies(df['outcome']) # Classification
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

In [173]:
df.groupby('outcome')['outcome'].count()

outcome
back.                 2203
buffer_overflow.        30
ftp_write.               8
guess_passwd.           53
imap.                   12
ipsweep.              1247
land.                   21
loadmodule.              9
multihop.                7
neptune.            107201
nmap.                  231
normal.              97278
perl.                    3
phf.                     4
pod.                   264
portsweep.            1040
rootkit.                10
satan.                1589
smurf.              280790
spy.                     2
teardrop.              979
warezclient.          1020
warezmaster.            20
Name: outcome, dtype: int64

## Training the neural network

In [174]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=19)

Train on 370515 samples, validate on 123506 samples
Epoch 1/19
370515/370515 - 29s - loss: 0.2516 - val_loss: 0.0515
Epoch 2/19
370515/370515 - 16s - loss: 0.0432 - val_loss: 0.0368
Epoch 3/19
370515/370515 - 15s - loss: 0.0315 - val_loss: 0.0296
Epoch 4/19
370515/370515 - 15s - loss: 0.0289 - val_loss: 0.0274
Epoch 5/19
370515/370515 - 15s - loss: 0.0286 - val_loss: 0.0260
Epoch 6/19
370515/370515 - 15s - loss: 0.0249 - val_loss: 0.0259
Epoch 7/19
370515/370515 - 15s - loss: 0.0272 - val_loss: 0.0228
Epoch 8/19
370515/370515 - 16s - loss: 0.0248 - val_loss: 0.0232
Epoch 9/19
370515/370515 - 15s - loss: 0.0215 - val_loss: 0.0221
Epoch 10/19
370515/370515 - 15s - loss: 0.0195 - val_loss: 0.0226
Epoch 11/19
370515/370515 - 16s - loss: 0.0234 - val_loss: 0.0215
Epoch 12/19
370515/370515 - 16s - loss: 0.0196 - val_loss: 0.0217
Epoch 13/19
370515/370515 - 16s - loss: 0.0240 - val_loss: 0.0228
Epoch 14/19
370515/370515 - 16s - loss: 0.0182 - val_loss: 0.0196
Epoch 15/19
370515/370515 - 15s -

In [175]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9980486777970301
